# advent of code 2024 - [day 7](https://adventofcode.com/2024/day/7)

### Parsing

In [ ]:
def gen_equations(file='input.txt'):
    """Generates tuples of integers"""
    file = open(file, 'r')
    for _, line in enumerate(file):
        els = tuple(line.split(":"))
        yield tuple((int(els[0]), tuple(int(x) for x in els[1].strip().split(' '))))

In [ ]:
file = "input.txt"
#file = "test.txt"

### Python solution

In [ ]:
from functools import cache

operators = {
    '*':lambda test_value, numbers, ops: test_value % numbers[-1] == 0 and is_valid_equation(test_value//numbers[-1], numbers[:-1], ops),
    '+':lambda test_value, numbers, ops: test_value - numbers[-1] >= 0 and is_valid_equation(test_value-numbers[-1], numbers[:-1], ops),
    '|':lambda test_value, numbers, ops: str(test_value)[1:].endswith(str(numbers[-1])) and is_valid_equation(int(str(test_value)[:-len(str(numbers[-1]))]), numbers[:-1], ops)
}

@cache
def is_valid_equation(test_value, numbers, ops):
    if len (numbers) == 1:
        return test_value == numbers[-1]
    else:
        return any(operators[k](test_value, numbers, ops) for k in operators if k in ops)
    
part1, part2 = (sum([x[0] if is_valid_equation(x[0], x[1], ops) else 0 for x in gen_equations(file)]) for ops in [('*','+'),('*','+','|')]) 
part1, part2